In [30]:
import pandas as pd


# Определение ставок VAT и duty
vat_rates = {
    'DE': 0.19,            # Germany
    'PT': 0.23,            # Portugal (mainland, Madeira, Azores)
    'ES': 0.21,            # Spain (except Canary Islands)
    'IE': 0.23,            # Ireland
    'SE': 0.25,            # Sweden
    'NL': 0.21,            # Netherlands
    'DK': 0.25,            # Denmark (mainland only)
    'FI': 0.255,           # Finland (except Åland Islands, raised in 2024)
    'IT': 0.22,            # Italy (except Livigno, Campione d’Italia, Lake Lugano waters)
    'AT': 0.20,            # Austria
    'BE': 0.21,            # Belgium
    'EE': 0.22             # Estonia (until July); use 0.24 for returns post-July 2025
}




In [31]:
df = pd.read_csv("TBdata.csv")

df = df[~df['Consignee Country'].isin(['IC', 'CH'])] # exclude all the non-eu countries

#fill all the missing MRN-values with Parcel ID:
df['MRN'] = df['MRN'].fillna(df['Parcel ID'])

# Ensure correct consignment grouping
df['Consignment Value'] = df.groupby('MRN')['Line Item Unit Price'].transform('sum')

df = df[df['Consignment Value'] <= 150]



In [32]:
# Calculate net quantity and price with VAT per row
df['Net Quantity'] = df['Line Item Quantity Imported'] - df['Line Item Quantity Returned']
df['VAT Rate'] = df['Consignee Country'].map(vat_rates)
df['Total Paid'] = df['Net Quantity'] * df['Line Item Unit Price']
df['VAT Paid'] = df['Net Quantity'] * df['Line Item Unit Price'] * df['VAT Rate']


# After mapping VAT rates
if df['VAT Rate'].isna().any():
    print("WARNING: Some countries missing VAT rates!")
    print(df[df['VAT Rate'].isna()]['Consignee Country'].unique())


# Group by country and sum the prices
result_df = df.groupby(['Consignee Country', 'VAT Rate']).agg({'VAT Paid': 'sum', 'Total Paid': 'sum'}).reset_index()
result_df.columns = ['Country', 'VAT Rate', 'Total VAT Paid', 'Total Items Price']

print(result_df)

result_df.to_excel('lv_declaration_data.xlsx', index=False)


   Country  VAT Rate  Total VAT Paid  Total Items Price
0       AT     0.200        194.8000             974.00
1       DE     0.190       2951.8400           15536.00
2       DK     0.250        278.2500            1113.00
3       EE     0.220         31.2400             142.00
4       ES     0.210        786.0300            3743.00
5       FI     0.255        553.0950            2169.00
6       IE     0.230       3821.6800           16616.00
7       IT     0.220        191.6200             871.00
8       NL     0.210       1837.2795            8748.95
9       PT     0.230        371.9100            1617.00
10      SE     0.250         82.2500             329.00
